# Proposta da POC

## Definição do projeto

## Requisitos

1. Criar base de dados com todas as geolocalizações das linhas que sejam de interesse do cliente.
2. Criar métricas para analisar a rota traçada pelos veículos.
3. Criar relatório com envio automático dos ofensores.

## Desenvolvimento

### Bibliotecas

In [10]:
from xml.etree import ElementTree as ET
import folium
from shapely.geometry import Point, LineString
import pandas as pd

### Paths

In [3]:
planned_trip_path = '/home/marcelo-borges/Documentos/Projetos/comparison_route/dados/1.1 LINHAS RMTC [2023-11-29].kml'
# trip_made_path = '/home/marcelo-borges/Documentos/Projetos/comparison_route/dados/20224.kml'

### Variáveis

In [4]:
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

### Funções

In [6]:

def read_and_extract_placemarks(file_path, namespace):
    """ 
    :param: file_path: caminho para o arquivo kml
    :return: lista de dicionários com os nomes e coordenadas dos placemarks

    A função lê um arquivo kml e extrai os placemarks, retornando uma lista de dicionários com os nomes e coordenadas dos placemarks.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    placemarks = []
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else "Unnamed"
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip() if placemark.find('.//kml:coordinates', namespace) is not None else "No coordinates"
        placemarks.append({'name': name, 'coordinates': coordinates})
    return placemarks


In [7]:
def parse_coordinates(coordinates_str):
    """
    param: coordinates_str: string de coordenadas (lat, long)
    return: lista de tuplas de coordenadas (long, lat)

    A função recebe uma string de coordenadas (lat, long) e retorna uma lista de tuplas de coordenadas (long, lat).
    """
    coords = coordinates_str.split()
    return [(float(coord.split(',')[1]), float(coord.split(',')[0])) for coord in coords]

### EDA

#### Dados previsto

##### Ingestão

In [46]:
placemarks_planned = read_and_extract_placemarks(planned_trip_path, ns)

In [47]:
df = pd.DataFrame(placemarks_planned)

##### Verificar dados nulos

##### Verificar duplicação

##### Verificar espaçamento

### Aplicação de regra de negócio

* Ajustando a separação para isolar corretamente o código, descrição e sentido


In [1]:
df['Código'] = df['name'].apply(lambda x: x.split('-', 1)[0])
df['Descrição e Sentido'] = df['name'].apply(lambda x: x.split(' - ', 1)[1] if ' - ' in x else '')
df['Descrição'] = df['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[0] if ' - ' in x else x)
df['Sentido'] = df['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[1] if ' - ' in x else '')
df.drop('Descrição e Sentido', axis=1, inplace=True)

NameError: name 'df' is not defined

##### Dados realizados

In [51]:
df.to_excel('geolocalizacao.xlsx', index=False)